In [1]:
from KnapsackNets import SafetyNet
import torch

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


# Generate Simulation Parameters

In [2]:
import numpy.random as npr
import numpy as np
import random 

global_seed = 0
random.seed(global_seed)
npr.seed(global_seed)

# category clusters
n_category_clusters = 50

# distribution of rate parameter for poisson process that generates demand for each cluster
n_demand_mean = 10
n_demand_stddev = 5

# Number of steps to run the poisson process for
n_steps = 40 # or 400

# rate paramter for possion distribution that models inventory for each category
n_inventory_min = 1
n_inventory_max = 20

# moments for normal distribution characterizing price
category_price_mean = 800
category_price_stddev = 400

# Simulations params for cancellation
a_mean = .5
a_stddev = .1

b_mean = -.2
b_stddev = .05

# DEMAND 
category_demand_rate = npr.normal(loc=n_demand_mean, scale=n_demand_stddev, size=n_category_clusters)
X_T = [np.random.poisson(np.clip(rate, a_min=0, a_max=None) , size=n_steps) for rate in category_demand_rate]
category_demand = np.array([np.sum(X) for X in X_T])

# AVG PRICE PER CATEGORY
category_price = npr.normal(loc=category_price_mean, scale=category_price_stddev, size=n_category_clusters)

# INVENTORY LEVEL
category_inventory_level_rate = npr.uniform(high=n_inventory_max, low=n_inventory_min, size=n_category_clusters)
category_inventory_level = npr.poisson(lam=category_inventory_level_rate)

# CANCELLATION PROBABILITY
a_param = npr.normal(loc=a_mean, scale=a_stddev, size=n_category_clusters)
b_param = npr.normal(loc=b_mean, scale=b_stddev, size=n_category_clusters)

category_cancel_prob = 1.0 - 1.0/(1.0+ np.exp(a_param + (b_param * category_inventory_level)))

# Store and Clip
params = {
    'inventory': category_inventory_level,
    'demand': category_demand,
    'price': category_price,
    'cancellation_probability': category_cancel_prob
}

for k,v in params.items():
    params[k] = np.clip(v, a_min=0, a_max=None)


# Initialize Safety Net

In [3]:
nCategories = 50
nThresholds = 20
temp_model = SafetyNet(
    nKnapsackCategories=nCategories, 
    nThresholds=nThresholds, 
    starting_thresholds=torch.ones(nCategories, nThresholds) # retail policy <- 10
)


In [4]:
from torch.nn import Parameter

starting_thresholds=torch.ones(1,nCategories)
thresholds_raw_matrix = Parameter(starting_thresholds)
thresholds_raw_matrix_norm= torch.div(thresholds_raw_matrix,torch.sum(thresholds_raw_matrix,dim=1).expand_as(thresholds_raw_matrix))
thresholds_raw_matrix_norm.size()

torch.Size([1, 50])

# Intialize Optimization/Estimation 

In [5]:
temp_optimizer_est = torch.optim.SGD([
{'params':[temp_model.prices_est]},
{'params':[temp_model.inventory_lam_est], 'lr': 1e-0},{'params':[temp_model.demand_distribution_est], 'lr': 1e-2},
{'params':[temp_model.cancel_coef_est], 'lr': 5e-3},
{'params':[temp_model.cancel_intercept_est], 'lr': 1e-1}
    ], lr=1e-7)
temp_optimizer_opt = torch.optim.SGD([
{'params':[temp_model.inventory_lam_opt], 'lr': 2e-5},{'params':[temp_model.demand_distribution_opt], 'lr': 1e-5},
{'params':[temp_model.cancel_coef_opt], 'lr': 2e-7},
{'params':[temp_model.cancel_intercept_opt], 'lr': 2e-6}
    ], lr=1e-8)
temp_optimizer_RHS =torch.optim.SGD([
{'params':[temp_model.cancel_rate_param, temp_model.accept_rate_param], 'lr': 1e-6}
    ], lr=1e-8)

In [6]:
from torch.autograd import Variable

In [22]:
nBatch = 32
temp_model.forward(
    category=Variable(torch.from_numpy(category_demand.reshape(1,nCategories).astype(np.float32)).expand(nBatch,nCategories)),
    inv_count=torch.from_numpy(category_inventory_level[:nThresholds].reshape(1,nThresholds).astype(np.float32)).expand(nBatch,nThresholds),
    price=torch.from_numpy(category_price[:nBatch].reshape(nBatch).astype(np.float32)).expand(nBatch),
    cancel=torch.from_numpy(category_cancel_prob[:nBatch].reshape(nBatch).astype(np.float32)).expand(nBatch),
    collection_thresholds=torch.ones(nBatch,nThresholds)*10
)

tensor(1.00000e+05 *
       [-8.6076, -8.6076, -8.6076, -8.6076, -8.6076, -8.6076, -8.6076,
        -8.6076, -8.6076, -8.6076, -8.6076, -8.6076, -8.6076, -8.6076,
        -8.6076, -8.6076, -8.6076, -8.6076, -8.6076, -8.6076, -8.6076,
        -8.6076, -8.6076, -8.6076, -8.6076, -8.6076, -8.6076, -8.6076,
        -8.6076, -8.6076, -8.6076, -8.6076])
torch.Size([32])
32
20
torch.Size([32, 20])


(tensor(-2.2921e+07),
 tensor(-1.2356e+09),
 tensor(-1.7796e+08),
 tensor([[ 1.2447e-02,  7.4681e-02,  1.6803e-01,  2.2404e-01,  2.1004e-01,
           1.5123e-01,  8.8216e-02,  4.3208e-02,  1.8228e-02,  6.7513e-03,
           2.2279e-03,  6.6288e-04,  1.7955e-04,  4.4650e-05,  1.0281e-05,
           2.2252e-06,  4.7784e-07,  1.2634e-07,  6.1810e-08,  5.2378e-08],
         [ 1.2447e-02,  7.4681e-02,  1.6803e-01,  2.2404e-01,  2.1004e-01,
           1.5123e-01,  8.8216e-02,  4.3208e-02,  1.8228e-02,  6.7513e-03,
           2.2279e-03,  6.6288e-04,  1.7955e-04,  4.4650e-05,  1.0281e-05,
           2.2252e-06,  4.7784e-07,  1.2634e-07,  6.1810e-08,  5.2378e-08],
         [ 1.2447e-02,  7.4681e-02,  1.6803e-01,  2.2404e-01,  2.1004e-01,
           1.5123e-01,  8.8216e-02,  4.3208e-02,  1.8228e-02,  6.7513e-03,
           2.2279e-03,  6.6288e-04,  1.7955e-04,  4.4650e-05,  1.0281e-05,
           2.2252e-06,  4.7784e-07,  1.2634e-07,  6.1810e-08,  5.2378e-08],
         [ 1.2447e-02,  7.4681e

In [ ]:
Variable(torch.from_numpy(category_demand.reshape(1,nCategories).astype(np.float32))).size(0)

In [ ]:
torch.ones(2,2,2).unsqueeze(3).size()

In [ ]:
torch.ones(2).unsqueeze(1).expand(2,2)

In [ ]:
torch.ones(1).unsqueeze(1)

In [ ]:
?torch.rand

In [ ]:
arrival_probability_batch_by_threshold_unnormed = torch.rand(nBatch, nThresholds)
arrival_probability_batch_by_threshold = torch.div(arrival_probability_batch_by_threshold_unnormed,
                                                   torch.sum(arrival_probability_batch_by_threshold_unnormed,dim=1).unsqueeze(1).expand_as(arrival_probability_batch_by_threshold_unnormed))

In [ ]:
arrival_probability_batch_by_threshold.size()

In [ ]:
torch.sum(arrival_probability_batch_by_threshold_unnormed,dim=1).unsqueeze(1).expand(nBatch, nThresholds).size()

In [ ]:
torch.sum(arrival_probability_batch_by_threshold_unnormed,dim=1).expand(32,2)

In [ ]:
torch.sum(arrival_probability_batch_by_threshold_unnormed,dim=1).unsqueeze(1)